In [1]:
%run ./MovieLens.ipynb

In [2]:
# 预测规则
def AF1(u, i):
    return r_u[u]

def AF2(u, i):
    return r_i[i]
    
def AF3(u, i):
    return r_u[u] / 2 + r_i[i] / 2

def AF4(u, i):
    return b_u[u] + r_i[i]

def AF5(u, i):
    return r_u[u] + b_i[i]

def AF6(u, i):
    return r_ + b_u[u] + b_i[i]

In [3]:
# 输出结果
predict(AF1, AF2, AF3, AF4, AF5, AF6)

RMSE: 1.0630, MAE: 0.8502
RMSE: 1.0334, MAE: 0.8276
RMSE: 0.9985, MAE: 0.8085
RMSE: 0.9593, MAE: 0.7557
RMSE: 0.9741, MAE: 0.7666
RMSE: 0.9620, MAE: 0.7604
